<a href="https://colab.research.google.com/github/hrushikesh-shelar/Cascade-Cup-Data-Science-Hackathon-My-Approach-/blob/main/Cascade_Cup_Achievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import train_test_split as tts

from xgboost import XGBClassifier

###Import Dataset & Pre-process





In [ ]:
# Import Dataset

df= pd.read_csv('https://raw.githubusercontent.com/i-am-hrushi/datasets/master/cascade_cup_train.csv')
df.head()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0,0.000000,0.200000,0.0,0.041667,0.025000,0.0,0.000000,0.175000,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0,0.008475,0.093220,0.0,0.012712,0.018644,0.0,0.084746,0.000000,0.033898,82.5,2
2,57867,6887426,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.341657,22.000000,83.051415,0,0.000000,0.002786,0.0,0.000000,0.000557,0.0,0.002786,0.000000,0.000000,0.0,1
3,272618,50742404,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.005735,3.000000,523.080000,0,0.000000,0.008403,0.0,0.000000,0.001681,0.0,0.000000,0.000000,0.008403,0.0,1
4,251123,45589200,2,2,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.456549,12.333333,53.766640,0,0.000000,0.204918,0.0,0.000000,0.049180,0.0,0.008197,0.057377,0.180328,0.0,1


In [ ]:
df.shape

(488877, 27)

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/i-am-hrushi/datasets/master/cascade_cup_test.csv')
test_df.head()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2
0,62683,33840667,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.033242,4.500000,120.313571,0,0.000000,0.062992,0.0,0.000000,0.012598,0.000000,0.000000,0.062992,0.000000,0.0
1,173104,1828421,2,1,0.001818,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.051057,9.500000,188.615500,0,0.000000,0.003636,0.0,0.000000,0.000727,0.000000,0.000000,0.003636,0.000000,0.0
2,306061,57353310,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.190984,4.675676,86.572800,0,0.000000,1.028571,0.0,0.000000,0.211429,0.000000,0.000000,0.742857,0.314286,0.0
3,422061,36618563,2,2,0.000000,0.0,0.0,4,0.0,0.0,0.0,0.130435,0.460543,21.142857,44.439050,0,0.007937,0.166667,0.0,0.067460,0.006349,0.000000,0.111111,0.023810,0.031746,201.0
4,219653,39484434,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.205251,2.689655,29.136643,0,0.000000,0.233871,0.0,0.116935,0.000000,0.233871,0.000000,0.000000,0.000000,0.0


In [ ]:
df.describe()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
count,488877.000000,4.888770e+05,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,4.888770e+05,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000
mean,271605.981815,4.236096e+07,1.975382,1.213037,0.082233,0.347520,0.403049,0.739748,0.000277,0.000981,0.012805,0.179148,0.341582,1.087395e+02,83.104654,0.320923,0.017065,0.391013,0.002009,0.074353,0.066927,0.032855,0.141705,0.150401,0.158382,164.765782,1.741643
std,156808.138002,1.821664e+07,0.430707,0.409454,2.467781,0.816334,0.925291,3.075954,0.012221,0.020381,0.159553,0.647588,0.195441,5.537516e+04,76.779586,7.649827,0.180402,0.905303,0.030561,0.211094,0.174791,0.193962,0.402612,0.451999,0.502065,981.988886,1.088559
min,0.000000,2.700000e+01,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000653,1.000000e+00,0.233000,0.000000,0.000000,0.000887,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,135779.000000,3.537599e+07,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199675,5.655172e+00,30.724000,0.000000,0.000000,0.040650,0.000000,0.000000,0.002265,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,271560.000000,4.336270e+07,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329743,8.500000e+00,62.501164,0.000000,0.000000,0.124031,0.000000,0.003968,0.015873,0.000000,0.011834,0.014563,0.015873,0.000000,1.000000
75%,407431.000000,5.370523e+07,2.000000,1.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150183,0.460492,1.308654e+01,112.246500,0.000000,0.009091,0.364486,0.000000,0.060000,0.059016,0.000000,0.117188,0.118644,0.122396,178.666667,2.000000
max,543196.000000,7.904203e+07,3.000000,2.000000,895.304000,4.000000,4.000000,624.000000,2.333333,3.000000,27.333333,262.666667,1.000000,3.826604e+07,7541.026000,3228.000000,63.388889,75.662281,8.196850,17.201754,18.756140,19.612903,27.905983,45.083333,55.153846,39304.000000,4.000000


In [ ]:
# Target Variable distribution

df.age_group.value_counts()

1    308315
4     60803
3     60404
2     59355
Name: age_group, dtype: int64

In [ ]:
# Under-sampling
df_1 = df[df['age_group']==1]
df_0 = df[df['age_group']!=1]

df_1_under = df_1.sample(60000)
df = pd.concat([df_1_under, df_0], axis = 0)

In [ ]:
# Target Variable distribution after undersapmling
df.age_group.value_counts()

4    60803
3    60404
1    60000
2    59355
Name: age_group, dtype: int64

In [ ]:
# Check for Null values
df.isnull().sum()

Unnamed: 0                         0
userId                             0
tier                               0
gender                             0
following_rate                     0
followers_avg_age                  0
following_avg_age                  0
max_repetitive_punc                0
num_of_hashtags_per_action         0
emoji_count_per_action             0
punctuations_per_action            0
number_of_words_per_action         0
avgCompletion                      0
avgTimeSpent                       0
avgDuration                        0
avgComments                        0
creations                          0
content_views                      0
num_of_comments                    0
weekends_trails_watched_per_day    0
weekdays_trails_watched_per_day    0
slot1_trails_watched_per_day       0
slot2_trails_watched_per_day       0
slot3_trails_watched_per_day       0
slot4_trails_watched_per_day       0
avgt2                              0
age_group                          0
d

In [ ]:
# Unique values for each column
for  i in df.columns:
    print(i,":\t\t\t\t\t", len(pd.unique(df[i])) )

Unnamed: 0 :					 240562
userId :					 240562
tier :					 3
gender :					 2
following_rate :					 13916
followers_avg_age :					 3085
following_avg_age :					 3224
max_repetitive_punc :					 86
num_of_hashtags_per_action :					 387
emoji_count_per_action :					 676
punctuations_per_action :					 2890
number_of_words_per_action :					 15555
avgCompletion :					 232948
avgTimeSpent :					 73684
avgDuration :					 222462
avgComments :					 181
creations :					 9522
content_views :					 27239
num_of_comments :					 3717
weekends_trails_watched_per_day :					 17551
weekdays_trails_watched_per_day :					 26681
slot1_trails_watched_per_day :					 10348
slot2_trails_watched_per_day :					 17115
slot3_trails_watched_per_day :					 18588
slot4_trails_watched_per_day :					 19136
avgt2 :					 53491
age_group :					 4


###Values Counts

In [ ]:
df.head()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
451504,471450,41339904,2,1,0.024194,3.0,3.0,0,0.0,0.0,0.0,0.3,0.245800,7.4,89.879429,0,0.016129,0.056452,0.0,0.016129,0.009677,0.000000,0.000000,0.048387,0.032258,234.000000,1
298241,58046,8957045,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.341657,3.5,83.051415,0,0.000000,0.010753,0.0,0.003584,0.000000,0.000000,0.000000,0.007168,0.000000,0.000000,1
166858,376586,74483739,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.366538,5.0,13.720000,0,0.000000,0.046512,0.0,0.000000,0.009302,0.046512,0.000000,0.000000,0.000000,0.000000,1
408656,361918,70999587,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.402952,12.5,136.315818,0,0.000000,0.200000,0.0,0.127273,0.000000,0.000000,0.000000,0.254545,0.000000,0.000000,1
285745,85220,32712184,2,1,0.007812,2.0,2.0,0,0.0,0.0,0.0,1.0,0.206872,8.5,138.111000,0,0.015625,0.039062,0.0,0.000000,0.012500,0.000000,0.007812,0.007812,0.046875,152.571429,1


In [ ]:
# One hot encoding of some columns
df = pd.get_dummies(df, columns=['gender','tier'], drop_first=True)
test_df = pd.get_dummies(test_df, columns = ['gender','tier'], drop_first=True)

In [ ]:
# Correlation with target feature
cor = df.corr()

index_li = (abs(cor.age_group).sort_values(ascending = False)[-10:]).index.to_list()

In [ ]:
abs(df.corr().age_group).sort_values(ascending = False)

age_group                          1.000000
following_avg_age                  0.199376
followers_avg_age                  0.198860
weekdays_trails_watched_per_day    0.180997
content_views                      0.177276
Unnamed: 0                         0.165442
slot4_trails_watched_per_day       0.160324
slot3_trails_watched_per_day       0.149170
weekends_trails_watched_per_day    0.137951
slot2_trails_watched_per_day       0.131160
slot1_trails_watched_per_day       0.098932
max_repetitive_punc                0.096198
avgt2                              0.085680
tier_2                             0.078492
creations                          0.071766
avgCompletion                      0.070145
number_of_words_per_action         0.066199
num_of_comments                    0.045023
userId                             0.037998
gender_2                           0.025799
avgComments                        0.023959
following_rate                     0.015095
tier_3                          

In [ ]:
index_li

['userId',
 'gender_2',
 'avgComments',
 'following_rate',
 'tier_3',
 'emoji_count_per_action',
 'punctuations_per_action',
 'num_of_hashtags_per_action',
 'avgDuration',
 'avgTimeSpent']

In [ ]:
df.columns.to_list()

['Unnamed: 0',
 'userId',
 'following_rate',
 'followers_avg_age',
 'following_avg_age',
 'max_repetitive_punc',
 'num_of_hashtags_per_action',
 'emoji_count_per_action',
 'punctuations_per_action',
 'number_of_words_per_action',
 'avgCompletion',
 'avgTimeSpent',
 'avgDuration',
 'avgComments',
 'creations',
 'content_views',
 'num_of_comments',
 'weekends_trails_watched_per_day',
 'weekdays_trails_watched_per_day',
 'slot1_trails_watched_per_day',
 'slot2_trails_watched_per_day',
 'slot3_trails_watched_per_day',
 'slot4_trails_watched_per_day',
 'avgt2',
 'age_group',
 'gender_2',
 'tier_2',
 'tier_3']

In [ ]:
df.avgDuration.describe()

count    240562.000000
mean         88.122978
std          75.684869
min           0.233000
25%          38.351545
50%          69.260424
75%         118.687537
max        7541.026000
Name: avgDuration, dtype: float64

In [ ]:
X, y = df.drop(columns=['age_group']), df['age_group']

### PCA

In [ ]:
# PCA for diamensionality reduction
pca = PCA(n_components= 15)

pca.fit_transform(X)
pca.transform(test_df)

array([[-4.84160057e+06,  2.24451863e+05, -5.23883790e+02, ...,
         5.54137650e-01,  2.38555129e-02,  7.98851400e-02],
       [-3.68536198e+07,  6.10176228e+04, -9.06961452e+02, ...,
         4.63652161e-01,  1.61182740e-02,  2.20261503e-02],
       [ 1.86714132e+07,  2.00121834e+04,  1.40005113e+02, ...,
         1.76231130e-01, -5.94545370e-02,  5.19021618e-01],
       ...,
       [ 2.97167376e+07, -1.27461467e+05,  5.17609060e+02, ...,
        -8.43555201e-02,  3.19755936e-01, -2.92612217e-01],
       [ 1.30270670e+07,  3.02887118e+05, -2.90265272e+02, ...,
        -2.37157233e-01,  1.71976530e-01, -9.21790334e-02],
       [ 1.23131632e+07,  1.72948784e+05, -1.47170671e+02, ...,
        -9.97170525e-01, -4.60724424e-02,  2.00121661e-02]])

In [ ]:
pca.explained_variance_ratio_

array([9.99891977e-01, 9.02681440e-05, 1.77499996e-05, 5.09129358e-09,
       1.32935823e-11, 3.42032341e-13, 3.79789315e-14, 3.07832571e-14,
       6.16754958e-15, 4.02389010e-15, 1.96058113e-15, 4.60778651e-16,
       3.99798062e-16, 3.85398673e-16, 2.93837429e-16])

###Feature Scaling


In [ ]:
#standardscaling the data before ML
ss = StandardScaler()
X_ss = ss.fit_transform(X)
test_df_ss = ss.transform(test_df)

X_ss = pd.DataFrame(data = X_ss, columns=X.columns)
test_df_ss = pd.DataFrame(data = test_df_ss, columns=test_df.columns)

###Model Implementation


In [ ]:
# train-test split
x_train,x_test,y_train,y_test = tts(X,y,test_size=0.25,random_state=42, stratify = y)

In [ ]:
# Applying RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 75, random_state=101, n_jobs= -1, min_samples_leaf=50, oob_score = True, max_features=None, criterion='gini' )
rfc.fit(x_train,y_train)

print(rfc)

y_pred_train = rfc.predict(x_train)
y_pred = rfc.predict(x_test)

print("train:\n", classification_report(y_train, y_pred_train))
print("test:\n", classification_report(y_test, y_pred))

print("train: ", f1_score(y_train, y_pred_train, average='weighted'))
print("test: ", f1_score(y_test, y_pred, average = 'weighted'))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=50, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=-1,
                       oob_score=True, random_state=101, verbose=0,
                       warm_start=False)
train:
               precision    recall  f1-score   support

           1       0.99      0.83      0.90     45000
           2       0.64      0.85      0.73     44516
           3       0.60      0.78      0.68     45303
           4       0.80      0.43      0.56     45602

    accuracy                           0.72    180421
   macro avg       0.76      0.72      0.72    180421
weighted avg       0.76      0.72      0.72    180421

test:
          

In [ ]:
#Applying GradientBoosting Classifier
gbc = GradientBoostingClassifier(n_estimators=100, random_state=101, max_depth=10, min_samples_split=5,learning_rate=0.5)

gbc.fit(x_train,y_train)

print(gbc)

y_pred_train = gbc.predict(x_train)
y_pred = gbc.predict(x_test)

print("train:\n", classification_report(y_train, y_pred_train))
print("test:\n", classification_report(y_test, y_pred))

print("train: ", f1_score(y_train, y_pred_train, average='weighted'))
print("test: ", f1_score(y_test, y_pred, average = 'weighted'))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.5, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=5,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=101, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
train:
               precision    recall  f1-score   support

           1       1.00      0.99      1.00     45000
           2       0.95      0.99      0.97     44516
           3       0.95      0.98      0.96     45303
           4       0.99      0.92      0.95     45602

    accuracy           

In [ ]:
# Applying XGBoost classiifer
xgb = XGBClassifier(n_estimators = 100, objective='multi:softprob', random_state=101, min_child_weight=30, scale_pos_weight=10, n_jobs=-1, reg_alpha = 1, reg_lambda=0)

xgb.fit(x_train,y_train)

print(xgb)

y_pred_train = xgb.predict(x_train)
y_pred = xgb.predict(x_test)

print("train:\n", classification_report(y_train, y_pred_train))
print("test:\n", classification_report(y_test, y_pred))

print("train: ", f1_score(y_train, y_pred_train, average='weighted'))
print("test: ", f1_score(y_test, y_pred, average = 'weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=30, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=101,
              reg_alpha=1, reg_lambda=0, scale_pos_weight=10, seed=None,
              silent=None, subsample=1, verbosity=1)
train:
               precision    recall  f1-score   support

           1       1.00      0.81      0.89     45000
           2       0.54      0.80      0.65     44516
           3       0.54      0.62      0.58     45303
           4       0.71      0.41      0.52     45602

    accuracy                           0.66    180421
   macro avg       0.70      0.66      0.66    180421
weighted avg       0.70      0.66      0.66    180421

test:
               precision    recall  f1-score   support

           1       1.

###Final Steps

In [ ]:
# Taking GradientBoostingClassifier for final prediction as it works the best
preds = gbc.predict(test_df)
preds = pd.DataFrame(data = preds.astype('int64'), columns=['prediction'])
preds.to_csv('submission.csv', index=False)